In [6]:
from config import FORVO_API_KEY
import asyncio
import aiohttp
import urllib.parse
import pandas as pd

In [7]:
df = pd.read_csv('basic_french_flashcards.csv', sep="&")
records = df.to_dict('records')

In [3]:
async def get_sound_file(dictionary, session, api_key = FORVO_API_KEY,):
    word = urllib.parse.quote(dictionary["french_word"])
    freq_idx = dictionary["frequency_idx"]
    url =  f"https://apifree.forvo.com/key/{api_key}/format/json/action/standard-pronunciation/word/{word}/language/fr"
    file_name = f"french_audio_{freq_idx}.mp3"
    path = "mp3_files/"
    try:
        async with session.get(url) as response:
            json_response = await response.json()
            mp3_download_url = json_response["items"][0]["pathmp3"]
        async with session.get(mp3_download_url) as mp3_response:
            mp3_file = await mp3_response.read()
        if sys.getsizeof(mp3_file) < 100:
            raise ValueError('No File Downloaded')
        with open(f'{path}{file_name}', 'wb') as local_file:
            local_file.write(mp3_file)
            print(f'Downloaded {freq_idx}')
    except Exception as e:
        return (dictionary, str(e))
    


async def get_multiple_sound_files(match_dict):
    all_responses = []
    async with aiohttp.ClientSession() as session:
        chunked_match_dicts = [match_dict[i:i + 5] for i in range(0, len(match_dict), 5)]
        for chunked_match_dict in chunked_match_dicts:
            tasks = []
            tasks.append(asyncio.sleep(5))
            for item in chunked_match_dict:
                tasks.append(get_sound_file(item, session))
            responses = await asyncio.gather(*tasks)
            all_responses.extend([response for response in responses if response is not None])
    return all_responses

In [4]:
# failed_downloads = await get_multiple_sound_files(records[3130:3380])

Downloaded 3134
Downloaded 3132
Downloaded 3133
Downloaded 3131
Downloaded 3135
Downloaded 3138
Downloaded 3137
Downloaded 3139
Downloaded 3136
Downloaded 3140
Downloaded 3141
Downloaded 3143
Downloaded 3142
Downloaded 3145
Downloaded 3144
Downloaded 3149
Downloaded 3150
Downloaded 3147
Downloaded 3148
Downloaded 3146
Downloaded 3151
Downloaded 3154
Downloaded 3155
Downloaded 3152
Downloaded 3153
Downloaded 3157
Downloaded 3156
Downloaded 3160
Downloaded 3159
Downloaded 3158
Downloaded 3162
Downloaded 3164
Downloaded 3165
Downloaded 3163
Downloaded 3161
Downloaded 3169
Downloaded 3170
Downloaded 3166
Downloaded 3167
Downloaded 3168
Downloaded 3174
Downloaded 3175
Downloaded 3172
Downloaded 3171
Downloaded 3173
Downloaded 3176
Downloaded 3178
Downloaded 3179
Downloaded 3180
Downloaded 3177
Downloaded 3181
Downloaded 3184
Downloaded 3183
Downloaded 3185
Downloaded 3182
Downloaded 3190
Downloaded 3188
Downloaded 3189
Downloaded 3187
Downloaded 3186
Downloaded 3192
Downloaded 3194
Download